In [1]:
import openai
import os
import fitz
from pandas.io.sql import table_exists
from pydantic import BaseModel
import json
from openpyxl import Workbook
import io
from API_info import ApiInfo
import pandas as pd

In [2]:
api_info = ApiInfo()

org, project = api_info.get_info()

client = openai.OpenAI(
    organization=org,
    project=project,
    api_key=os.environ.get('MY_OPENAI_KEY')
)

# Configurar a chave da OpenAI (pode ser via variável de ambiente)
OPENAI_API_KEY = os.getenv("MY_OPENAI_KEY")

In [3]:
def extract_text_from_pdf(pdf_bytes: bytes) -> str:
    doc = fitz.open(stream=pdf_bytes, filetype="pdf")
    text = "\n".join(page.get_text("text") for page in doc)
    return text.strip()

In [4]:
def get_relevant_info_from_openai(text: str) -> str:
    prompt = f"""
    Você é um assistente inteligente especializado em processar documentos com informações delicados, então é melhor pender para o excesso do que cortar informações importantes.
    Analise o seguinte texto e extraia as informações relevante.
    Não poupe detalhes, mas seja conciso, gostaria de ter todas as informações relevantes desse documento então é necessário procurar pelo maximo de informações relevantes possiveis.
    Evite repetição de informações, busque sempre informações novas.
    Retorne um resumo breve e uma lista de pontos-chave.
    Mantenha a resposta limpa, ou seja, somente retorne o conteúdo do JSON, nada mais.

    Texto do documento:
    {text}

    Responda no seguinte formato JSON:
    {{
        "summary": "<resumo do conteúdo>",
        "key_points": ["<ponto 1>", "<ponto 2>", "<ponto 3>", ...]
    }}
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": prompt}],
        max_tokens=5000,
    )

    extracted_data = response.choices[0].message.content
    print(extracted_data)
    return extracted_data

In [13]:
def create_excel_from_analysis(json_data: str):
    prompt = f"""
    Você é um assistente inteligente especializado em criar tabelas no Excel.
    Analise o seguinte JSON com informações chave e resumo, e crie um JSON que será tratado e transformado em uma tabela excel.
    Dessa forma, organize o JSON já em mente que ele será transformado em uma tabela.
    Mantenha a resposta limpa, ou seja, somente retorne o conteúdo do JSON, nada mais.

    JSON:
    {json_data}


    Responda no seguinte formato JSON:
        {{
            "data": [
                {{ "Ponto Chave": "<pontoChave1>", "Valor": "<valor1>" }},
                {{ "Ponto Chave": "<pontoChave2>", "Valor": "<valor2>" }},
                ...
            ]
        }}
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": prompt}],
        max_tokens=5000,
    )

    response_content = response.choices[0].message.content

    print(response_content)


    if not response_content:
        raise ValueError("Empty response from OpenAI API")

    try:
        response_json = json.loads(response_content)
        print(response_json.get("data"))
    except json.JSONDecodeError as e:
        raise ValueError(f"Invalid JSON response: {e}")

    table = pd.DataFrame(response_json.get("data", []))

    return table

In [7]:
pdf_info = extract_text_from_pdf(open("pdf/Fatura_unlocked.pdf", "rb").read())
print(pdf_info)

Resumo da fatura
Olá, Heitor! A sua fatura chegou!
Total da sua fatura
R$ 517,45
Este é o valor que você precisa pagar nesse mês
Limite de crédito total
R$ 520,00
Data de Vencimento
28/02/2025
Pagamento mínimo: R$ 77,62
O pagamento mínimo é o valor que você precisa pagar nesse mês para não ficar em atraso com a fatura.
A diferença será cobrada com juros e impostos na próxima fatura.
Encargos em caso de pagamento mínimo desta fatura
Pagamento mínimo: R$ 77,62
Encargos rotativos
15,60% am
IOF diário
0,246% am
IOF adicional
0,380% am
Em caso de pagamento mínimo o valor financiado será:
Valor total financiado
R$ 439,83
Total a pagar em encargos e IOF do
rotativo
R$ 85,17
Total a pagar
R$ 525,00
Pagamento via Boleto
Caso o pagamento da fatura seja feito via boleto, o prazo para
reconhecimento do pagamento e liberação do limite é de até 3 dias úteis.
07790.00116 01001.305299 88953.553364 5 00000000000000
AUTENTICAÇÃO MECÂNICA FICHA DE COMPENSAÇÃO
Pagamento via Pix
Faça o pagamento pela conta

In [8]:
relevant_info = get_relevant_info_from_openai(pdf_info)

{
    "summary": "Heitor Saulo Dantas Santos recebeu sua fatura com um total a pagar de R$ 517,45, com data de vencimento em 28/02/2025. O limite total do cartão é de R$ 520,00, sendo que R$ 517,45 já foram utilizados, restando R$ 2,55 disponíveis. O pagamento mínimo para evitar inadimplência é de R$ 77,62, e se optar pelo pagamento mínimo, o saldo remanescente será financiado com juros e encargos. Diversas opções de parcelamento da fatura estão disponíveis, e o IOF e juros estão incluídos nos valores. O cartão será bloqueado em caso de falta de pagamento, com encargos adicionais aplicáveis ao saldo remanescente. Instruções para o pagamento da fatura via boleto ou Pix também foram fornecidas.",
    "key_points": [
        "Total da fatura: R$ 517,45",
        "Data de vencimento: 28/02/2025",
        "Limite total do cartão: R$ 520,00",
        "Pagamento mínimo: R$ 77,62",
        "Encargos rotativos: 15,60% ao mês",
        "IOF diário: 0,246% ao mês",
        "Valor total financiado

In [14]:
table = create_excel_from_analysis(relevant_info)

{
    "data": [
        { "Ponto Chave": "Total da fatura", "Valor": "R$ 517,45" },
        { "Ponto Chave": "Data de vencimento", "Valor": "28/02/2025" },
        { "Ponto Chave": "Limite total do cartão", "Valor": "R$ 520,00" },
        { "Ponto Chave": "Pagamento mínimo", "Valor": "R$ 77,62" },
        { "Ponto Chave": "Encargos rotativos", "Valor": "15,60% ao mês" },
        { "Ponto Chave": "IOF diário", "Valor": "0,246% ao mês" },
        { "Ponto Chave": "Valor total financiado se pagar mínimo", "Valor": "R$ 439,83" },
        { "Ponto Chave": "Total de encargos e IOF", "Valor": "R$ 85,17" },
        { "Ponto Chave": "Pagamentos podem ser feitos via", "Valor": "boleto ou Pix" },
        { "Ponto Chave": "Opções de parcelamento disponíveis", "Valor": "1 até 14 parcelas" },
        { "Ponto Chave": "Multa por atraso", "Valor": "2% sobre saldo não pago" },
        { "Ponto Chave": "Juros de mora", "Valor": "1% ao mês aplicado diariamente" },
        { "Ponto Chave": "Cartão será bl

In [30]:
for index, row_num in table.iterrows():
    print(f"Index: {index}, Row: {row_num}")

Index: 0, Row: Ponto Chave    Total da fatura
Valor                R$ 517,45
Name: 0, dtype: object
Index: 1, Row: Ponto Chave    Data de vencimento
Valor                  28/02/2025
Name: 1, dtype: object
Index: 2, Row: Ponto Chave    Limite total do cartão
Valor                       R$ 520,00
Name: 2, dtype: object
Index: 3, Row: Ponto Chave    Pagamento mínimo
Valor                  R$ 77,62
Name: 3, dtype: object
Index: 4, Row: Ponto Chave    Encargos rotativos
Valor               15,60% ao mês
Name: 4, dtype: object
Index: 5, Row: Ponto Chave       IOF diário
Valor          0,246% ao mês
Name: 5, dtype: object
Index: 6, Row: Ponto Chave    Valor total financiado se pagar mínimo
Valor                                       R$ 439,83
Name: 6, dtype: object
Index: 7, Row: Ponto Chave    Total de encargos e IOF
Valor                         R$ 85,17
Name: 7, dtype: object
Index: 8, Row: Ponto Chave    Pagamentos podem ser feitos via
Valor                            boleto ou Pix
Name:

In [39]:
# Add the summary to the DataFrame
def add_summary_to_table(summary: str, table: pd.DataFrame) -> pd.DataFrame:
    summary_row = pd.DataFrame([{"Ponto Chave": "Resumo", "Valor": summary}])
    table_with_summary = pd.concat([summary_row, table], ignore_index=True)
    return table_with_summary

# Apply formatting and save the DataFrame to an Excel file
def save_table_to_excel(table: pd.DataFrame, output_path: str):
    with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
        table.to_excel(writer, index=False, sheet_name='Análise de Dados')

        workbook = writer.book
        worksheet = writer.sheets['Análise de Dados']

        # Set the column width and format
        for column in table:
            column_width = max(table[column].astype(str).map(len).max(), len(column))
            col_idx = table.columns.get_loc(column)
            worksheet.set_column(col_idx, col_idx, column_width)

        # Add a header format
        header_format = workbook.add_format({
            'bold': True,
            'text_wrap': True,
            'valign': 'top',
            'fg_color': '#D7E4BC',
            'border': 1
        })

        # Write the column headers with the defined format
        for col_num, value in enumerate(table.columns.values):
            worksheet.write(0, col_num, value, header_format)

        # Add a format for the summary row
        summary_format = workbook.add_format({
            'bold': True,
            'text_wrap': True,
            'valign': 'top',
            'fg_color': '#FFEB9C',
            'border': 1
        })

        # Apply the summary format to the first row
        for col_num in range(len(table.columns)):
            worksheet.write(1, col_num, table.iloc[0, col_num], summary_format)

        # Apply the general format to the rest of the rows, starting from the third row
        general_format = workbook.add_format({
            'text_wrap': True,
            'valign': 'top',
            'border': 1
        })

        for row_num in range(1, len(table)):
            for col_num in range(len(table.columns)):
                worksheet.write(row_num+1, col_num, table.iloc[row_num, col_num], general_format)

In [24]:
relevant_info_json = json.loads(relevant_info)
relevant_info_json['summary']

'Heitor Saulo Dantas Santos recebeu sua fatura com um total a pagar de R$ 517,45, com data de vencimento em 28/02/2025. O limite total do cartão é de R$ 520,00, sendo que R$ 517,45 já foram utilizados, restando R$ 2,55 disponíveis. O pagamento mínimo para evitar inadimplência é de R$ 77,62, e se optar pelo pagamento mínimo, o saldo remanescente será financiado com juros e encargos. Diversas opções de parcelamento da fatura estão disponíveis, e o IOF e juros estão incluídos nos valores. O cartão será bloqueado em caso de falta de pagamento, com encargos adicionais aplicáveis ao saldo remanescente. Instruções para o pagamento da fatura via boleto ou Pix também foram fornecidas.'

In [40]:
table_with_summary = add_summary_to_table(relevant_info_json['summary'], table)
save_table_to_excel(table_with_summary, "static/output_with_summary.xlsx")